In [ ]:
! pip install -U langgraph langsmith
! pip install langchain_community
! pip install langchain_openai

In [3]:
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END ,state
from langgraph.graph.message import add_messages
from langchain_core.messages import HumanMessage,AIMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
import os

In [ ]:
# Configuration de la clé API
os.environ['OPENAI_API_KEY'] = "sk-pro c5xEwhMZOsYA6xKN4HruNnPRcA"



# Création du modèle LLM
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# Contrastive Prompting (CoT): Learning from Mistakes to Improve LLMs

Contrastive Prompting (Contrastive Chain-of-Thought), introduced by Chia et al. (2023), is an ingenious method that enriches the learning of language models (LLMs) by showing them both correct and incorrect reasoning.

## How Does Contrastive Prompting Work?

Imagine you're teaching a student. Instead of only showing examples of correct answers, you also deliberately present common reasoning errors so they learn to avoid them.

Contrastive Prompting follows this pedagogical logic in two key steps:

1. **Demonstration of contrasts**: The LLM is provided with examples that contain both incorrect and correct reasoning chains for the same problem.

2. **Identification of pitfalls**: These contrastive examples allow the model to recognize typical errors, false reasoning, and logical traps to avoid.

 This approach leverages a fundamental principle of human learning: we often learn better from our mistakes than from our successes. By explicitly exposing the model to erroneous reasoning, it's allowed to develop a form of "critical discernment."

Domain  such as:
* Arithmetic reasoning
* Factual questions
* Complex problem solving

It's like teaching a student not only the correct method for solving an equation but also the common errors that lead to false solutions.

 

In [7]:
contrastive_cot_prompt_template = ChatPromptTemplate.from_messages([
    ("system", """
    Generate Contrastive Chain-of-Thought (CoT) Prompting based on user questions.
    
    Contrastive CoT Prompting is an advanced form of Chain-of-Thought that includes both correct and incorrect reasoning paths to help the model distinguish between valid and invalid approaches.
    
    This technique significantly improves performance on arithmetic reasoning and factual QA tasks by demonstrating both how to reason correctly and how not to reason.
    
    The Contrastive CoT process includes the following steps:
    1. Problem understanding: Clearly state the problem and identify key information.
    2. Incorrect reasoning path: Show a plausible but flawed reasoning process that leads to an incorrect answer.
    3. Error identification: Identify the specific mistakes or logical errors in the incorrect approach.
    4. Correct reasoning path: Demonstrate the proper reasoning process step by step.
    5. Final answer with justification: Provide the correct answer with clear justification.
    
    Please structure your response in the following format:
    
    Step 1: [Problem understanding]
    Step 2: [Incorrect reasoning path]
    Step 3: [Error identification]
    Step 4: [Correct reasoning path]
    Step 5: [Final answer with justification]
    
    Example:
    Question: "A store sells notebooks for $2.50 each. If you buy 6 notebooks, how much change will you receive from a $20 bill?"
    
    Step 1: Problem understanding:
    I need to calculate how much change I'll receive after buying 6 notebooks at $2.50 each with a $20 bill. This requires figuring out the total cost of the notebooks and then subtracting that from $20.
    
    Step 2: Incorrect reasoning path:
    To find the cost of 6 notebooks, I'll multiply:
    6 × $2.50 = $12.50
    Then the change would be:
    $20 - $12.50 = $7.50
    So I would receive $7.50 in change.
    
    Step 3: Error identification:
    Wait, I made a calculation error. When multiplying 6 by $2.50:
    6 × $2.50 = 6 × 2.5 = 12 + 3 = $15.00, not $12.50
    The error was in the multiplication step, where I incorrectly calculated 6 × $2.50 as $12.50.
    
    Step 4: Correct reasoning path:
    Let's solve this correctly:
    Cost of one notebook = $2.50
    Cost of 6 notebooks = 6 × $2.50 = $15.00
    I give the cashier a $20 bill
    Change = $20.00 - $15.00 = $5.00
    
    Step 5: The correct answer is $5.00 in change. This is justified because 6 notebooks at $2.50 each costs $15.00 in total (6 × $2.50 = $15.00), and when paying with a $20 bill, the change is $20.00 - $15.00 = $5.00.
    """),
    ("human", "{question}"),
    ("assistant", """
    Step 1: Problem understanding:
    [Clearly state the problem and identify the key information, variables, and what is being asked. Make sure to highlight any relevant details or constraints.]
    
    Step 2: Incorrect reasoning path:
    [Present a plausible but flawed approach to solving the problem:
    - Show reasonable-looking steps that someone might actually follow
    - Carry the reasoning through to an incorrect conclusion
    - Make the mistake subtle enough to be educational (not an obvious arithmetic error)]
    
    Step 3: Error identification:
    [Identify and explain the specific errors in the incorrect reasoning:
    - Pinpoint exactly where and how the reasoning went wrong
    - Explain why this approach is problematic or leads to an incorrect result
    - Clarify any misconceptions demonstrated in the incorrect approach]
    
    Step 4: Correct reasoning path:
    [Demonstrate the proper way to solve the problem:
    1. [First step in the correct solution process]
    2. [Second step in the correct solution process]
    3. [Continue with additional steps as needed]
    - Ensure each step is logical and builds on previous steps
    - Show all work clearly]
    
    Step 5: [State the final answer clearly and concisely, followed by a justification that highlights why this approach leads to the correct solution and how it avoids the pitfalls identified in Step 3]
    """)
])

For Contrastive Chain-of-Thought (CoT) Prompting, the most suitable questions are those where distinguishing between correct and incorrect reasoning paths significantly enhances model performance. Here are types of questions that would be particularly well-suited to this approach:

### Mathematical and Arithmetic Problems
- Word problems with common calculation errors
- Multi-step calculations where order of operations matters
- Percentage, ratio, and fraction problems
- Financial calculations with multiple variables

### Logical Reasoning Tasks
- Deductive reasoning problems with potential fallacies
- Questions involving causal relationships
- Syllogisms and formal logic problems
- Problems requiring avoidance of common cognitive biases

### Factual Question Answering
- Questions with commonly confused facts or entities
- Multi-part factual questions requiring verification at each step
- Questions where precision in terminology matters
- Historical or scientific questions with common misconceptions

### Concrete Examples of Questions

1. "If a shirt originally costs $80 and is discounted by 25%, then has a 'buy one get one half off' promotion applied when purchasing two, what is the average price per shirt?"
   - Showcasing correct vs. incorrect discount application order

2. "A baker has 3/4 cup of flour and needs 2 1/2 cups. How much more flour does the baker need?"
   - Demonstrating proper vs. improper fraction subtraction

3. "If all dolphins are mammals, and all mammals are warm-blooded, what valid conclusion can we draw about dolphins?"
   - Contrasting valid logical deduction vs. invalid logical fallacies

4. "The population of a bacteria culture doubles every 30 minutes. If we start with 100 bacteria, how many will we have after 3 hours?"
   - Showing exponential growth calculation vs. linear calculation errors

5. "Who was the first person to reach the South Pole, and in what year did this occur?"
   - Contrasting accurate factual recall vs. common historical confusion

Contrastive CoT Prompting is particularly effective for these types of questions because:

- It explicitly demonstrates both correct reasoning and common pitfalls
- It helps the model recognize and avoid typical reasoning errors
- It builds an internal "error detector" for flawed reasoning paths
- It enhances discrimination between superficially similar but conceptually different approaches
- It mimics effective teaching strategies that highlight both what to do and what to avoid

This approach is especially powerful for problems with common, identifiable error patterns where seeing the contrast between correct and incorrect reasoning provides more instructional value than seeing only correct solutions. The method effectively teaches models to recognize and avoid reasoning traps, much like how humans learn from both successes and failures.

In [10]:
import pandas as pd
import re
from typing import Optional, TypedDict, Annotated
from langchain.schema import AIMessage, HumanMessage

# Function to generate the Contrastive Chain of Thought prompt and extract the final answer (Step 5)
def generate_contrastive_cot_node(state):
    question = state['messages'][-1].content  # Get the last question
    prompt_value = contrastive_cot_prompt_template.invoke({"question": question})
    messages = prompt_value.to_messages()
    response = llm.invoke(messages)
    
    # Keep the full response for processing
    full_response = response.content
    
    # Extract the final answer from Step 5
    final_answer_pattern = r'Step 5:([\s\S]*?)(?=$|Step \d:)'
    final_answer_match = re.search(final_answer_pattern, full_response)
    
    if final_answer_match:
        final_solution = final_answer_match.group(1).strip()
    else:
        # Alternative pattern
        alternative_pattern = r'Step 5: \[Final answer with justification\]([\s\S]*?)(?=$|Step \d:)'
        alternative_match = re.search(alternative_pattern, full_response)
        
        if alternative_match:
            final_solution = alternative_match.group(1).strip()
        else:
            final_solution = "Final solution not found in the expected format."
    
    # Save both the question, final answer, and full processing to a text file
    with open('contrastive_cot_result.txt', 'w', encoding='utf-8') as f:
        f.write("=== QUESTION ===\n\n")
        f.write(question)
        f.write("\n\n=== FINAL ANSWER (STEP 5) ===\n\n")
        f.write(final_solution)
        f.write("\n\n=== COMPLETE PROCESSING (FOR REFERENCE) ===\n\n")
        f.write(full_response)
    
    # Return only the final answer in messages, plus the original question for reference
    return {
        "messages": [AIMessage(content=final_solution)],
        "question": question,
        "full_response": full_response
    }

# Definition of state
class State(TypedDict):
    messages: Annotated[list, add_messages]
    question: Optional[str]
    full_response: Optional[str]

# Graph creation
graph_builder = StateGraph(State)
graph_builder.add_node("generate_contrastive_cot", generate_contrastive_cot_node)

# Graph configuration
graph_builder.set_entry_point("generate_contrastive_cot")
graph_builder.add_edge("generate_contrastive_cot", END)
graph = graph_builder.compile()

# Usage example
inputs = {
    "messages": [HumanMessage(content="Who was the first person to reach the South Pole, and in what year did this occur?")],
    "question": None,
    "full_response": None
}

# Print the question at the beginning
print("QUESTION:")
print(inputs["messages"][0].content)
print("\nFINAL ANSWER (STEP 5):")

# Graph execution - only display Step 5
original_question = ""
for output in graph.stream(inputs):
    for key, value in output.items():
        if key == "generate_contrastive_cot":
            messages = value['messages']
            for message in messages:
                if isinstance(message, AIMessage):
                    print(message.content)
            
            # Store question for reference
            if "question" in value:
                original_question = value["question"]

# Print confirmation that results were saved
print("\n(Complete results saved in 'contrastive_cot_result.txt')")

QUESTION:
Who was the first person to reach the South Pole, and in what year did this occur?

FINAL ANSWER (STEP 5):
The correct answer is Roald Amundsen, who was the first person to reach the South Pole on December 14, 1911. This is justified because historical records clearly indicate that Amundsen's expedition arrived at the South Pole before Scott's, making him the first to achieve this milestone.

(Complete results saved in 'contrastive_cot_result.txt')
